In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


# ChatOpenAI 인스턴스 생성
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


# IMDb에서 가져온 영화 정보 예시
movie_examples = [
    {
        "title": "The Shawshank Redemption",
        "director": "Frank Darabont",
        "cast": "Tim Robbins, Morgan Freeman, Bob Gunton",
        "budget": "$25 million",
        "box_office": "$58.3 million",
        "genre": "Drama",
        "synopsis": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",
    },
    {
        "title": "The Godfather",
        "director": "Francis Ford Coppola",
        "cast": "Marlon Brando, Al Pacino, James Caan",
        "budget": "$6 million",
        "box_office": "$245 million",
        "genre": "Crime, Drama",
        "synopsis": "The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.",
    },
    {
        "title": "The Dark Knight",
        "director": "Christopher Nolan",
        "cast": "Christian Bale, Heath Ledger, Aaron Eckhart",
        "budget": "$185 million",
        "box_office": "$1.005 billion",
        "genre": "Action, Crime, Drama",
        "synopsis": "When the menace known as The Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.",
    },
    {
        "title": "Forrest Gump",
        "director": "Robert Zemeckis",
        "cast": "Tom Hanks, Robin Wright, Gary Sinise",
        "budget": "$55 million",
        "box_office": "$678.2 million",
        "genre": "Drama, Romance",
        "synopsis": "The presidencies of Kennedy and Johnson, the events of Vietnam, Watergate, and other historical events unfold from the perspective of an Alabama man with an IQ of 75, whose only desire is to be reunited with his childhood sweetheart.",
    },
    # Add more movie examples here
]


example_template = examples = [
    ("human", "Tell me about the movie {title}."),
    (
        "ai",
        "{director} directed {title}, starring {cast}. It belongs to the {genre} genre. The budget was {budget} and it grossed {box_office}. Here's a synopsis: {synopsis}",
    ),
]

# exmple prompt template
example_template = (
    "Human: Tell me about the {title}.\n"
    "AI: {director} directed {title}, starring {cast}. "
    "It belongs to the {genre} genre. The budget was {budget} and it grossed {box_office}. "
    "Here's a synopsis: {synopsis}"
)


# exmple prompt template
movie_prompt = PromptTemplate.from_template(example_template)


# fewshot template
prompt = FewShotPromptTemplate(
    example_prompt=movie_prompt,
    examples=movie_examples,
    suffix="Human: Tell me know other movie {title}",
    input_variables=["title"],
)


# 대화 체인 생성
chain = prompt | chat

# 영화 정보 요청
chain.invoke({"title": "The God Father"})

AI: The Godfather is a classic film, but there are also two sequels: The Godfather Part II and The Godfather Part III. The Godfather Part II follows the story of Vito Corleone as a young man and his rise to power, while The Godfather Part III focuses on an aging Michael Corleone as he tries to legitimize his family's business.

AIMessage(content="AI: The Godfather is a classic film, but there are also two sequels: The Godfather Part II and The Godfather Part III. The Godfather Part II follows the story of Vito Corleone as a young man and his rise to power, while The Godfather Part III focuses on an aging Michael Corleone as he tries to legitimize his family's business.", response_metadata={'finish_reason': 'stop'})

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 인스턴스 생성
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


# IMDb에서 가져온 영화 정보 예시
movie_examples = [
    {
        "title": "The Shawshank Redemption",
        "answer": """
        "director": "Frank Darabont",
        "cast": "Tim Robbins, Morgan Freeman, Bob Gunton",
        "budget": "$25 million",
        "box_office": "$58.3 million",
        "genre": "Drama",
        "synopsis": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",
        """,
    },
    {
        "title": "The Godfather",
        "answer": """
        "director": "Francis Ford Coppola",
        "cast": "Marlon Brando, Al Pacino, James Caan",
        "budget": "$6 million",
        "box_office": "$245 million",
        "genre": "Crime, Drama",
        "synopsis": "The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.",
        """,
    },
    {
        "title": "The Dark Knight",
        "answer": """
        "director": "Christopher Nolan",
        "cast": "Christian Bale, Heath Ledger, Aaron Eckhart",
        "budget": "$185 million",
        "box_office": "$1.005 billion",
        "genre": "Action, Crime, Drama",
        "synopsis": "When the menace known as The Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.",
        """,
    },
    {
        "title": "Forrest Gump",
        "answer": """
        "director": "Robert Zemeckis",
        "cast": "Tom Hanks, Robin Wright, Gary Sinise",
        "budget": "$55 million",
        "box_office": "$678.2 million",
        "genre": "Drama, Romance",
        "synopsis": "The presidencies of Kennedy and Johnson, the events of Vietnam, Watergate, and other historical events unfold from the perspective of an Alabama man with an IQ of 75, whose only desire is to be reunited with his childhood sweetheart.",
        """,
    },
]
output_parser = StrOutputParser()

# ChatPromptTemplate 생성
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {title}?"),
        ("ai", "{answer}"),
    ]
)

# FewShotChatMessagePromptTemplate 생성
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=movie_examples,
)

# 최종 ChatPromptTemplate 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {title}?"),
    ]
)


# 대화 체인 생성
chain = final_prompt | chat | output_parser

# 영화 정보 요청
chain.invoke({"title": "Forrest Gump"})

"Forrest Gump" is a 1994 American film directed by Robert Zemeckis and starring Tom Hanks in the title role. It follows the life of Forrest Gump, a man with a low IQ but a kind heart, as he inadvertently influences several major historical events in the 20th century. The film won multiple Academy Awards, including Best Picture and Best Actor for Tom Hanks.

'"Forrest Gump" is a 1994 American film directed by Robert Zemeckis and starring Tom Hanks in the title role. It follows the life of Forrest Gump, a man with a low IQ but a kind heart, as he inadvertently influences several major historical events in the 20th century. The film won multiple Academy Awards, including Best Picture and Best Actor for Tom Hanks.'